In [1]:
%reload_ext autoreload
%autoreload 2

In [ ]:
from llama.workflows.finetune import finetune

finetune(
    task='direct',
    data_path='/home/tbai4/llama3/dumps/simple/from_mad.json',
    ckpt_dir='/scratch4/jeisner1/tjbai/llama_8b',
    tokenizer_path='/scratch4/jeisner1/tjbai/llama_8b/tokenizer.model',
    output_dir='/scratch4/jeisner1/tjbai/checkpoints/direct/from_mad',
    max_seq_len=8*8192,
    epochs=4,
    gradient_accumulation_steps=4,
    checkpoint_freq=100,
    validation_freq=100,
    lora_rank=64,
    lora_alpha=32,
    lora_dropout=0.05,
    learning_rate=5e-5,
)

In [2]:
import os
from llama import Workflow, Llama
from llama.util import find_free_port, load_ckpt

os.environ["RANK"] = "0"
os.environ["WORLD_SIZE"] = "1"
os.environ["MASTER_ADDR"] = "localhost"
os.environ["MASTER_PORT"] = str(find_free_port())

workflow = Workflow.build(
    ckpt_dir='/scratch4/jeisner1/tjbai/llama_8b',
    tokenizer_path='/scratch4/jeisner1/tjbai/llama_8b/tokenizer.model',
    max_seq_len=2*8192,
    max_batch_size=1,
    model_parallel_size=1,
    max_nodes=100,
    use_lora=True,
    lora_rank=64,
    lora_alpha=32,
    lora_dropout=0.05,
)

llama = Llama(workflow.model, workflow.tokenizer)

/home/tbai4/llama3/llama/util.py:60: UserWarning: 16384 does not lie within [1, 8192]
  warnings.warn(f"{max_seq_len} does not lie within [1, 8192]")


> initializing model parallel with size 1
> initializing ddp with size 1
> initializing pipeline with size 1
Converting to LoRA
Loaded in 10.63 seconds


In [7]:
import json
from llama.workflows.tot import load_math_problems, eval_solutions

llama.model.reshape_cache(4)

with open('/home/tbai4/llama3/dumps/math_baseline_direct_test.json') as f:
    data = json.load(f)
    print(len(data))

problems = load_math_problems('/home/tbai4/llama3/data/MATH', split='test')[:500]

outputs = eval_solutions(
    llama,
    solutions=data,
    problems=problems
)

sum(outputs)

500


100%|██████████| 500/500 [03:15<00:00,  2.55it/s]


94

In [ ]:
load_ckpt(workflow, '/scratch4/jeisner1/tjbai/checkpoints/direct/from_madpar/lora_step-149.pt')

In [5]:
from llama.workflows.tot import load_math_problems

problems = load_math_problems('/home/tbai4/llama3/data/MATH', split='test')

In [20]:
from llama.workflows.simple import math_direct

math_direct(
    workflow,
    problem=problems[1]['problem'],
    temperature=0.7,
)

'Answer: Based on the summary, it appears that all three agents agree that the correct answer is $\\boxed{2}$. This is because they all recognize that the statement "At least one of the numbers $a, b,$ and $c$ is odd" is equivalent to the statement "At least one of $a, b,$ and $c$ is odd or all of $a, b,$ and $c$ are even".\n\nThe first agent\'s statement that the answer is $\\boxed{2}$ is correct, as it is the number of possibilities where all three numbers are even. The second agent\'s statement that the answer is $\\boxed{3}$ is incorrect, as it is the number of possibilities where at least one of the numbers is odd, not all three.\n\nThe third agent\'s statement that the answer is $\\boxed{2}$ is also correct, as it is the number of possibilities where all three numbers are even. This is because the statement "At least one of the numbers $a, b,$ and $c$ is odd" is equivalent to the statement "At least one of $a, b,$ and $c$ is odd or all of $a, b,$ and $c$ are even".\n\nTherefore, 

In [17]:
outputs

"Answer: Based on the summary, it's clear that all three agents agree on the final answer to the math problem, which is $441$.\n\nHere is the revised answer:\n\nThe final answer is: $\\boxed{441}$"

In [10]:
from llama.workflows.trainers import DirectTrainer, ListDataset

dataset = ListDataset('/home/tbai4/llama3/dumps/simple/from_tot.json')

trainer = DirectTrainer(workflow, '/scratch4/jeisner1/tjbai/checkpoints/simple/from_mad', 1e-5)

Training 54.5M / 8.1B parameters


In [15]:
import numpy as np
import torch
from tqdm import tqdm

with torch.no_grad():
    total = 0
    n = 0
    for i, sample in tqdm(enumerate(dataset[:100])):
        if len(sample['outputs']['solution']) == 0:
            continue
        n += 1
        total += trainer.step(sample)[0].cpu()
    print(total / n)

100it [00:29,  3.37it/s]

tensor(0.8235, dtype=torch.float32)
